# Signal Replay Analysis

This notebook enables fast risk parameter optimization by replaying existing signal traces.

**Key Features:**
- Load pre-computed signal traces from global store
- Optimize position sizing strategies
- Test different stop loss and profit target levels
- Portfolio construction and allocation
- Fast iteration without signal regeneration

In [ ]:
# Parameters will be injected here by papermill
# This cell is tagged with 'parameters' for papermill to recognize it
run_dir = "."
config_name = "config"
symbols = ["SPY"]
timeframe = "5m"

# Risk optimization parameters
initial_capital = 100000
max_position_size = 0.1  # Max 10% per position
max_portfolio_risk = 0.02  # Max 2% portfolio risk
position_sizing_methods = ["fixed", "volatility", "kelly"]

# Stop loss and profit target grid
stop_loss_levels = [0.01, 0.02, 0.03, 0.05, 0.075, 0.1, 0.15, 0.2]
profit_target_levels = [0.02, 0.03, 0.05, 0.075, 0.1, 0.15, 0.2, 0.3]

# Portfolio construction
max_concurrent_positions = 5
rebalance_frequency = "daily"  # daily, weekly, monthly
correlation_threshold = 0.7  # Max correlation between strategies

# Analysis options
use_global_traces = True  # Use global trace store
analyze_top_n = 20  # Analyze top N strategies from signal analysis

## Setup

In [ ]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import json
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Configure plotting
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

# Convert run_dir to Path
run_dir = Path(run_dir).resolve()
print(f"Analyzing run: {run_dir.name}")
print(f"Full path: {run_dir}")
print(f"Config: {config_name}")
print(f"Symbol(s): {symbols}")
print(f"Timeframe: {timeframe}")

# Import TraceStore if using global traces
if use_global_traces:
    import sys
    # Find project root
    for parent in run_dir.parents:
        if (parent / 'src' / 'analytics').exists():
            if str(parent) not in sys.path:
                sys.path.insert(0, str(parent))
            break
    
    from src.analytics.trace_store import TraceStore
    trace_store = TraceStore()
    print(f"\n✅ Connected to global trace store")
    print(f"   Available strategies: {len(trace_store.list_strategies())}")

## Load Strategy Recommendations

In [ ]:
# Load recommendations from signal analysis
recommendations_path = run_dir / 'optimized_recommendations.json'
if recommendations_path.exists():
    with open(recommendations_path, 'r') as f:
        recommendations = json.load(f)
    
    print(f"✅ Loaded recommendations from signal analysis")
    print(f"   Best strategy: {recommendations['best_optimized']['strategy_type']}")
    print(f"   Sharpe: {recommendations['best_optimized']['optimal_sharpe']:.2f}")
    print(f"   Stop: {recommendations['best_optimized']['optimal_stop_pct']:.3f}%")
    print(f"   Target: {recommendations['best_optimized']['optimal_target_pct']:.3f}%")
    
    # Extract strategy hashes to analyze
    strategy_hashes = [recommendations['best_optimized']['strategy_hash']]
    for strategy in recommendations['top_10_optimized'][:analyze_top_n-1]:
        strategy_hashes.append(strategy['strategy_hash'])
    
    print(f"\n📊 Will analyze {len(strategy_hashes)} strategies")
else:
    print("❌ No recommendations found. Run signal_analysis.ipynb first!")
    strategy_hashes = []

## Load Signal Traces

In [ ]:
# Load traces for selected strategies
if use_global_traces and strategy_hashes:
    print("Loading traces from global store...")
    traces = trace_store.load_traces(strategy_hashes)
    print(f"✅ Loaded {len(traces)} traces")
    
    # Get metadata for each strategy
    strategy_metadata = {}
    for hash_val in strategy_hashes:
        metadata = trace_store.get_strategy_metadata(hash_val)
        if metadata:
            strategy_metadata[hash_val] = metadata
else:
    # Load from workspace
    print("Loading traces from workspace...")
    traces = {}
    strategy_metadata = {}
    
    # Load strategy index
    strategy_index = pd.read_parquet(run_dir / 'strategy_index.parquet')
    
    for hash_val in strategy_hashes:
        strategy_info = strategy_index[strategy_index['strategy_hash'] == hash_val]
        if not strategy_info.empty:
            trace_path = run_dir / strategy_info.iloc[0]['trace_path']
            if trace_path.exists():
                traces[hash_val] = pd.read_parquet(trace_path)
                strategy_metadata[hash_val] = strategy_info.iloc[0].to_dict()

print(f"\n📊 Trace Summary:")
for hash_val, trace in traces.items():
    meta = strategy_metadata.get(hash_val, {})
    print(f"  {meta.get('strategy_type', 'unknown')} ({hash_val[:8]}): {len(trace)} signals")

## Load Market Data

In [ ]:
# Load market data for position sizing and risk calculations
market_data = None
for symbol in symbols:
    try:
        # Try different possible locations
        data_paths = [
            run_dir / f'data/{symbol}_{timeframe}.csv',
            run_dir / f'{symbol}_{timeframe}.csv',
            run_dir.parent / f'data/{symbol}_{timeframe}.csv',
            Path(f'/Users/daws/ADMF-PC/data/{symbol}_{timeframe}.csv')
        ]
        
        for data_path in data_paths:
            if data_path.exists():
                market_data = pd.read_csv(data_path)
                market_data['timestamp'] = pd.to_datetime(market_data['timestamp'])
                market_data = market_data.sort_values('timestamp')
                
                # Calculate additional metrics for risk
                market_data['returns'] = market_data['close'].pct_change()
                market_data['volatility_20'] = market_data['returns'].rolling(20).std()
                market_data['atr'] = ((market_data['high'] - market_data['low']).rolling(14).mean())
                market_data['atr_pct'] = market_data['atr'] / market_data['close']
                
                print(f"✅ Loaded market data from: {data_path}")
                print(f"   Date range: {market_data['timestamp'].min()} to {market_data['timestamp'].max()}")
                print(f"   Total bars: {len(market_data)}")
                break
        
        if market_data is not None:
            break
            
    except Exception as e:
        print(f"Error loading data for {symbol}: {e}")

if market_data is None:
    print("❌ Could not load market data")

## Position Sizing Functions

In [ ]:
def calculate_position_size(method, capital, price, volatility=None, win_rate=None, 
                          avg_win=None, avg_loss=None, max_size=0.1):
    """
    Calculate position size based on different methods.
    
    Args:
        method: 'fixed', 'volatility', or 'kelly'
        capital: Current capital
        price: Entry price
        volatility: 20-day volatility (for volatility sizing)
        win_rate: Historical win rate (for Kelly)
        avg_win: Average winning return (for Kelly)
        avg_loss: Average losing return (for Kelly)
        max_size: Maximum position size as fraction of capital
    
    Returns:
        Number of shares to buy
    """
    if method == 'fixed':
        # Fixed percentage of capital
        position_value = capital * max_size
        shares = int(position_value / price)
        
    elif method == 'volatility':
        # Size inversely proportional to volatility
        if volatility and volatility > 0:
            # Target 1% portfolio volatility
            target_vol = 0.01
            position_pct = min(target_vol / volatility, max_size)
            position_value = capital * position_pct
            shares = int(position_value / price)
        else:
            # Fallback to fixed if no volatility
            shares = int(capital * max_size / price)
            
    elif method == 'kelly':
        # Kelly criterion
        if win_rate and avg_win and avg_loss and avg_loss != 0:
            # Kelly formula: f = (p*b - q)/b
            # where p = win_rate, q = 1-win_rate, b = avg_win/abs(avg_loss)
            b = avg_win / abs(avg_loss)
            kelly_pct = (win_rate * b - (1 - win_rate)) / b
            
            # Apply Kelly fraction (typically 0.25) and max size
            kelly_pct = max(0, min(kelly_pct * 0.25, max_size))
            position_value = capital * kelly_pct
            shares = int(position_value / price)
        else:
            # Fallback to fixed
            shares = int(capital * max_size / price)
    else:
        # Default to fixed
        shares = int(capital * max_size / price)
    
    return max(0, shares)


def apply_risk_management(trace_df, market_data, stop_pct, target_pct, 
                         position_sizing='fixed', initial_capital=100000):
    """
    Apply position sizing and risk management to signal trace.
    
    Returns:
        DataFrame with portfolio equity curve and metrics
    """
    # Merge trace with market data
    trace_df = trace_df.copy()
    trace_df['ts'] = pd.to_datetime(trace_df['ts'])
    
    # Create portfolio tracking
    portfolio = {
        'cash': initial_capital,
        'positions': {},
        'equity_curve': [],
        'trades': []
    }
    
    # Track performance for Kelly sizing
    trade_returns = []
    
    # Process each bar
    for idx, bar in market_data.iterrows():
        current_equity = portfolio['cash']
        
        # Update position values
        for symbol, position in portfolio['positions'].items():
            position['current_price'] = bar['close']
            position['value'] = position['shares'] * bar['close']
            current_equity += position['value']
            
            # Check stop loss
            if stop_pct > 0:
                if position['direction'] == 1:  # Long
                    stop_price = position['entry_price'] * (1 - stop_pct/100)
                    if bar['low'] <= stop_price:
                        # Stop hit - close position
                        exit_price = stop_price
                        returns = (exit_price - position['entry_price']) / position['entry_price']
                        portfolio['cash'] += position['shares'] * exit_price
                        trade_returns.append(returns)
                        del portfolio['positions'][symbol]
                        continue
            
            # Check profit target
            if target_pct > 0:
                if position['direction'] == 1:  # Long
                    target_price = position['entry_price'] * (1 + target_pct/100)
                    if bar['high'] >= target_price:
                        # Target hit - close position
                        exit_price = target_price
                        returns = (exit_price - position['entry_price']) / position['entry_price']
                        portfolio['cash'] += position['shares'] * exit_price
                        trade_returns.append(returns)
                        del portfolio['positions'][symbol]
                        continue
        
        # Check for new signals
        bar_signals = trace_df[trace_df['idx'] == idx]
        if not bar_signals.empty:
            signal = bar_signals.iloc[0]
            
            # Position sizing parameters
            if len(trade_returns) > 10 and position_sizing == 'kelly':
                winning_returns = [r for r in trade_returns if r > 0]
                losing_returns = [r for r in trade_returns if r <= 0]
                win_rate = len(winning_returns) / len(trade_returns)
                avg_win = np.mean(winning_returns) if winning_returns else 0
                avg_loss = np.mean(losing_returns) if losing_returns else -0.01
            else:
                win_rate = avg_win = avg_loss = None
            
            # Calculate position size
            shares = calculate_position_size(
                method=position_sizing,
                capital=current_equity,
                price=bar['close'],
                volatility=bar.get('volatility_20'),
                win_rate=win_rate,
                avg_win=avg_win,
                avg_loss=avg_loss,
                max_size=max_position_size
            )
            
            if shares > 0 and signal['val'] != 0:
                # Check if we have enough cash
                position_cost = shares * bar['close']
                if position_cost <= portfolio['cash']:
                    # Open position
                    portfolio['cash'] -= position_cost
                    portfolio['positions'][signal['sym']] = {
                        'shares': shares,
                        'entry_price': bar['close'],
                        'entry_time': bar['timestamp'],
                        'direction': 1 if signal['val'] > 0 else -1,
                        'current_price': bar['close'],
                        'value': position_cost
                    }
        
        # Record equity
        portfolio['equity_curve'].append({
            'timestamp': bar['timestamp'],
            'equity': current_equity,
            'cash': portfolio['cash'],
            'positions': len(portfolio['positions'])
        })
    
    # Create equity curve DataFrame
    equity_df = pd.DataFrame(portfolio['equity_curve'])
    equity_df['returns'] = equity_df['equity'].pct_change()
    
    return equity_df, trade_returns

## Risk Parameter Optimization

In [ ]:
# Test different risk parameters for each strategy
optimization_results = []

for hash_val, trace in traces.items():
    meta = strategy_metadata.get(hash_val, {})
    print(f"\nOptimizing {meta.get('strategy_type', 'unknown')} ({hash_val[:8]})...")
    
    # Test each combination of parameters
    for stop in stop_loss_levels:
        for target in profit_target_levels:
            for sizing in position_sizing_methods:
                # Run backtest with these parameters
                equity_df, trade_returns = apply_risk_management(
                    trace, market_data, stop, target, sizing, initial_capital
                )
                
                if len(equity_df) > 0:
                    # Calculate metrics
                    total_return = (equity_df['equity'].iloc[-1] / initial_capital - 1)
                    
                    # Sharpe ratio
                    if equity_df['returns'].std() > 0:
                        sharpe = equity_df['returns'].mean() / equity_df['returns'].std() * np.sqrt(252)
                    else:
                        sharpe = 0
                    
                    # Max drawdown
                    cummax = equity_df['equity'].expanding().max()
                    drawdown = (equity_df['equity'] / cummax - 1)
                    max_dd = drawdown.min()
                    
                    # Store results
                    optimization_results.append({
                        'strategy_hash': hash_val,
                        'strategy_type': meta.get('strategy_type', 'unknown'),
                        'stop_loss': stop,
                        'profit_target': target,
                        'position_sizing': sizing,
                        'total_return': total_return,
                        'sharpe_ratio': sharpe,
                        'max_drawdown': max_dd,
                        'num_trades': len(trade_returns),
                        'win_rate': len([r for r in trade_returns if r > 0]) / len(trade_returns) if trade_returns else 0,
                        'final_equity': equity_df['equity'].iloc[-1]
                    })

# Convert to DataFrame
risk_optimization_df = pd.DataFrame(optimization_results)
print(f"\n✅ Tested {len(risk_optimization_df)} parameter combinations")

## Find Optimal Parameters

In [ ]:
# Find best parameters for each strategy
print("\n🏆 Optimal Risk Parameters by Strategy")
print("=" * 100)

best_params = risk_optimization_df.loc[risk_optimization_df.groupby('strategy_hash')['sharpe_ratio'].idxmax()]

for _, row in best_params.iterrows():
    print(f"\n{row['strategy_type']} ({row['strategy_hash'][:8]}):")
    print(f"  Best Sharpe: {row['sharpe_ratio']:.2f}")
    print(f"  Parameters: Stop={row['stop_loss']:.3f}%, Target={row['profit_target']:.3f}%, Sizing={row['position_sizing']}")
    print(f"  Total Return: {row['total_return']*100:.1f}%")
    print(f"  Max Drawdown: {row['max_drawdown']*100:.1f}%")
    print(f"  Win Rate: {row['win_rate']*100:.1f}%")
    print(f"  Trades: {row['num_trades']}")

# Overall best configuration
best_overall = risk_optimization_df.nlargest(1, 'sharpe_ratio').iloc[0]
print(f"\n\n🥇 BEST OVERALL CONFIGURATION:")
print(f"Strategy: {best_overall['strategy_type']} ({best_overall['strategy_hash'][:8]})")
print(f"Sharpe Ratio: {best_overall['sharpe_ratio']:.2f}")
print(f"Stop Loss: {best_overall['stop_loss']:.3f}%")
print(f"Profit Target: {best_overall['profit_target']:.3f}%")
print(f"Position Sizing: {best_overall['position_sizing']}")
print(f"Total Return: {best_overall['total_return']*100:.1f}%")
print(f"Final Equity: ${best_overall['final_equity']:,.2f}")

## Position Sizing Analysis

In [ ]:
# Compare position sizing methods
sizing_comparison = risk_optimization_df.groupby('position_sizing').agg({
    'sharpe_ratio': ['mean', 'std', 'max'],
    'total_return': ['mean', 'std', 'max'],
    'max_drawdown': ['mean', 'min'],
    'win_rate': 'mean'
})

print("\n📊 Position Sizing Method Comparison")
print("=" * 80)
print(sizing_comparison)

# Visualize results
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Sharpe by sizing method
ax = axes[0, 0]
risk_optimization_df.boxplot(column='sharpe_ratio', by='position_sizing', ax=ax)
ax.set_title('Sharpe Ratio by Position Sizing Method')
ax.set_xlabel('Position Sizing')
ax.set_ylabel('Sharpe Ratio')

# Return by sizing method
ax = axes[0, 1]
risk_optimization_df.boxplot(column='total_return', by='position_sizing', ax=ax)
ax.set_title('Total Return by Position Sizing Method')
ax.set_xlabel('Position Sizing')
ax.set_ylabel('Total Return')

# Drawdown by sizing method
ax = axes[1, 0]
risk_optimization_df.boxplot(column='max_drawdown', by='position_sizing', ax=ax)
ax.set_title('Max Drawdown by Position Sizing Method')
ax.set_xlabel('Position Sizing')
ax.set_ylabel('Max Drawdown')

# Win rate by sizing method
ax = axes[1, 1]
risk_optimization_df.boxplot(column='win_rate', by='position_sizing', ax=ax)
ax.set_title('Win Rate by Position Sizing Method')
ax.set_xlabel('Position Sizing')
ax.set_ylabel('Win Rate')

plt.tight_layout()
plt.show()

## Portfolio Construction

In [ ]:
# Select strategies for portfolio based on correlation
print("\n🏗️ Building Optimal Portfolio")
print("=" * 80)

# Get top strategies by Sharpe
top_strategies = best_params.nlargest(max_concurrent_positions * 2, 'sharpe_ratio')

# Calculate correlation matrix of returns
strategy_returns = {}
for _, strategy in top_strategies.iterrows():
    trace = traces[strategy['strategy_hash']]
    equity_df, _ = apply_risk_management(
        trace, market_data, 
        strategy['stop_loss'], 
        strategy['profit_target'],
        strategy['position_sizing'], 
        initial_capital
    )
    strategy_returns[strategy['strategy_hash']] = equity_df['returns'].values

# Create correlation matrix
returns_df = pd.DataFrame(strategy_returns)
correlation_matrix = returns_df.corr()

# Select uncorrelated strategies
selected_strategies = []
available = list(top_strategies['strategy_hash'].values)

# Start with highest Sharpe
selected_strategies.append(available[0])
available.remove(available[0])

# Add strategies with low correlation
while len(selected_strategies) < max_concurrent_positions and available:
    min_corr = 1.0
    best_candidate = None
    
    for candidate in available:
        # Check max correlation with selected strategies
        max_corr = max([abs(correlation_matrix.loc[candidate, s]) for s in selected_strategies])
        if max_corr < min_corr and max_corr < correlation_threshold:
            min_corr = max_corr
            best_candidate = candidate
    
    if best_candidate:
        selected_strategies.append(best_candidate)
        available.remove(best_candidate)
    else:
        break

print(f"Selected {len(selected_strategies)} strategies for portfolio:")
for i, hash_val in enumerate(selected_strategies):
    strategy_info = best_params[best_params['strategy_hash'] == hash_val].iloc[0]
    print(f"{i+1}. {strategy_info['strategy_type']} - Sharpe: {strategy_info['sharpe_ratio']:.2f}")

# Display correlation heatmap
plt.figure(figsize=(10, 8))
selected_corr = correlation_matrix.loc[selected_strategies, selected_strategies]
sns.heatmap(selected_corr, annot=True, cmap='coolwarm', center=0, 
            xticklabels=[s[:8] for s in selected_strategies],
            yticklabels=[s[:8] for s in selected_strategies])
plt.title('Portfolio Strategy Correlations')
plt.show()

## Simulate Portfolio Performance

In [ ]:
# Simulate portfolio with selected strategies
portfolio_equity = initial_capital
allocation_per_strategy = 1.0 / len(selected_strategies)
portfolio_curve = []

# Run each strategy with allocated capital
strategy_curves = {}
for hash_val in selected_strategies:
    strategy_info = best_params[best_params['strategy_hash'] == hash_val].iloc[0]
    trace = traces[hash_val]
    
    equity_df, _ = apply_risk_management(
        trace, market_data,
        strategy_info['stop_loss'],
        strategy_info['profit_target'],
        strategy_info['position_sizing'],
        initial_capital * allocation_per_strategy
    )
    
    strategy_curves[hash_val] = equity_df

# Combine equity curves
timestamps = strategy_curves[selected_strategies[0]]['timestamp']
portfolio_equity_curve = pd.DataFrame({'timestamp': timestamps})
portfolio_equity_curve['portfolio_value'] = 0

for hash_val, curve in strategy_curves.items():
    portfolio_equity_curve['portfolio_value'] += curve['equity'].values

portfolio_equity_curve['returns'] = portfolio_equity_curve['portfolio_value'].pct_change()

# Calculate portfolio metrics
portfolio_return = (portfolio_equity_curve['portfolio_value'].iloc[-1] / initial_capital - 1)
portfolio_sharpe = portfolio_equity_curve['returns'].mean() / portfolio_equity_curve['returns'].std() * np.sqrt(252)
cummax = portfolio_equity_curve['portfolio_value'].expanding().max()
drawdown = (portfolio_equity_curve['portfolio_value'] / cummax - 1)
portfolio_max_dd = drawdown.min()

print(f"\n📊 PORTFOLIO PERFORMANCE")
print("=" * 80)
print(f"Total Return: {portfolio_return*100:.1f}%")
print(f"Sharpe Ratio: {portfolio_sharpe:.2f}")
print(f"Max Drawdown: {portfolio_max_dd*100:.1f}%")
print(f"Final Value: ${portfolio_equity_curve['portfolio_value'].iloc[-1]:,.2f}")

# Plot portfolio performance
fig, axes = plt.subplots(2, 1, figsize=(15, 10))

# Equity curve
ax = axes[0]
ax.plot(portfolio_equity_curve['timestamp'], portfolio_equity_curve['portfolio_value'], 
        label='Portfolio', linewidth=2, color='black')
for i, (hash_val, curve) in enumerate(strategy_curves.items()):
    ax.plot(curve['timestamp'], curve['equity'], 
            label=f"Strategy {i+1}", alpha=0.5, linewidth=1)
ax.set_title('Portfolio Equity Curve')
ax.set_xlabel('Date')
ax.set_ylabel('Equity ($)')
ax.legend()
ax.grid(True, alpha=0.3)

# Drawdown
ax = axes[1]
ax.fill_between(portfolio_equity_curve['timestamp'], drawdown * 100, 0, 
                alpha=0.3, color='red')
ax.set_title('Portfolio Drawdown')
ax.set_xlabel('Date')
ax.set_ylabel('Drawdown (%)')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Export Optimized Configuration

In [ ]:
# Export optimal risk parameters and portfolio configuration
risk_config = {
    'generated_at': datetime.now().isoformat(),
    'initial_capital': initial_capital,
    'best_individual_strategy': {
        'strategy_hash': best_overall['strategy_hash'],
        'strategy_type': best_overall['strategy_type'],
        'stop_loss': float(best_overall['stop_loss']),
        'profit_target': float(best_overall['profit_target']),
        'position_sizing': best_overall['position_sizing'],
        'expected_sharpe': float(best_overall['sharpe_ratio']),
        'expected_return': float(best_overall['total_return'])
    },
    'portfolio_configuration': {
        'strategies': [],
        'allocation_method': 'equal_weight',
        'max_correlation': float(correlation_threshold),
        'expected_sharpe': float(portfolio_sharpe),
        'expected_return': float(portfolio_return),
        'expected_max_drawdown': float(portfolio_max_dd)
    }
}

# Add portfolio strategies
for hash_val in selected_strategies:
    strategy_info = best_params[best_params['strategy_hash'] == hash_val].iloc[0]
    risk_config['portfolio_configuration']['strategies'].append({
        'strategy_hash': hash_val,
        'strategy_type': strategy_info['strategy_type'],
        'stop_loss': float(strategy_info['stop_loss']),
        'profit_target': float(strategy_info['profit_target']),
        'position_sizing': strategy_info['position_sizing'],
        'allocation': float(allocation_per_strategy)
    })

# Save configuration
with open(run_dir / 'risk_optimized_config.json', 'w') as f:
    json.dump(risk_config, f, indent=2)

# Save detailed results
risk_optimization_df.to_csv(run_dir / 'risk_optimization_results.csv', index=False)

print("\n✅ Configuration exported:")
print(f"  - risk_optimized_config.json")
print(f"  - risk_optimization_results.csv")
print(f"\n🚀 Ready for live trading with optimized risk parameters!")